#### PlantGrowth R dataset project
##### Problem statement
Complete the project in a single notebook called project.ipynb in your repository. The same style should be used as detailed above: explanations in MarkDown and code comments, clean code, and regular commits. Use plots as appropriate.

In this project, you will analyze the PlantGrowth R dataset. You will find a short description of it on Vicent Arel-Bundock's Rdatasets page. The dataset contains two main variables, a treatment group and the weight of plants within those groups.

Your task is to perform t-tests and ANOVA on this dataset while describing the dataset and explaining your work. In doing this you should:

- Download and save the dataset to your repository.

- Describe the data set in your notebook.

- Describe what a t-test is, how it works, and what the assumptions are.

- Perform a t-test to determine whether there is a significant difference between the two treatment groups trt1 and trt2.

- Perform ANOVA to determine whether there is a significant difference between the three treatment groups ctrl, trt1, and trt2.

- Explain why it is more appropriate to apply ANOVA rather than several t-tests when analyzing more than two groups.



##### Introduction

![Lady Tasting Tea](images/venusflytrap.PNG)
<br>
<br>
This project involves an analysis of the PlantGrowth dataset from Vincent Arel Bundock's [R Rdatasets collection](https://vincentarelbundock.github.io/Rdatasets/). The dataset includes two variables: plant weight and treatment groups. My objective is to examine any potential differences in plant growth across these groups using t-tests and ANOVA. The analysis will (hopefully) help us to understand more about the impact of growth treatments on plant weight.

We'll start off by important the relevant libraries. 















In [22]:
import pandas as pd

We need to read in the csv and display the dataframe information. The info() method provides information about the dataset, including the number of rows, columns, column names, data types, and memory usage.

In [23]:
# Let's load the .csv into a dataframe and get the df info
PGdata = pd.read_csv(r'C:\Users\Administrator\Data Analytics\Applied Statistics\PlantGrowth.csv') 
PGdata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rownames  30 non-null     int64  
 1   weight    30 non-null     float64
 2   group     30 non-null     object 
dtypes: float64(1), int64(1), object(1)
memory usage: 852.0+ bytes


This helps to confirm that treatment is a categorical variable and weight is a numerical variable. This confirmation is important in the planning stages, since both the t-test and the ANOVA assume specific data types. We can see that there are 30 non-null entries so no we know the sample size and since the values are non-null, we don't need to worry about any missing values.